**извлечение признаков из текста на естественном языке**

токенизация и очистка текста


Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
np.set_printoptions(precision=2) # вывод на печать до 2 знака
import pandas as pd
import re
import gzip

In [2]:
pd.options.display.max_colwidth = 200  

## загружаем тексты

In [3]:
# загружаем тексты
df = pd.read_pickle('../data/text/news.pkl.gz')
print('записей:',len(df))

записей: 3196


In [4]:
df.sample(5)

,text,tag
1368,"Ученые нашли различие в зрении мужчин и женщин\n\n30 ноября 2016 в 10:32\n\n42.TUT.BY\n\nЖенщины и мужчины смотрят на лица и интерпретируют визуальную информацию по-разному, что может приводить к ...",tech
605,Как перестать есть слишком много сладкого: 7 практических советов\n\n9 декабря 2016 в 8:40\n\nkitchenmag.ru\n\nСахар сам по себе не представляет никакой опасности для организма. В умеренных количе...,woman
1349,"Космический корабль ""Прогресс"" разрушился, обломки упали на территории Тувы\n\nопубликовано: 1 декабря 2016 в 19:09\n\nобновлено: 1 декабря 2016 в 21:02\n\nТАСС / РБК\n\nРоскосмос сообщил о потере...",tech
1447,"Трамп решил обнародовать свои планы по уходу из бизнеса в январе\n\nДональд Трамп решил отложить до января свою первую пресс-конференцию в качестве избранного президента США, на которой он собирае...",politics
109,"ЦСКА переиграл болгарский клуб ""Левски"" со счетом 2:1 В рамках розыгрыша\nКубка УЕФА московский ЦСКА не без труда переиграл болгарский клуб ""Левски""\nсо счетом 2:1...",sport


## токенизация и очистка

In [5]:
# with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
#     stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )

# print('количество стоп-слов:',len(stopwords))
# sorted(stopwords)

# from Stemmer import Stemmer
# pacman -S python-pystemmer
# pip install pystemmer

In [6]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords

stopwords = set(nltk_stopwords.words('russian') )

In [7]:
%%time 

df['text_clean'] = df['text'].str.lower() # приведение в lowercase

# замена символов-разделителей (-,_) на пробел
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'_', ' ', s))

# замена цифр
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' ', s))

# делим строки на слова (токенизация)
df['text_clean'] = df['text_clean'].apply(lambda t: [ w.strip() for w in t.split() if len(w.strip())>2 ] )

# удаление лишних слов 
df['text_clean'] = df['text_clean'].apply(lambda t:[w for w in t if w not in stopwords])

# стемминг, выделение основы слова
# df['text_clean'] = df['text_clean'].apply( lambda t: Stemmer('russian').stemWords(t) )
df['text_clean'] = df['text_clean'].apply(lambda t:[ SnowballStemmer('russian').stem(w) for w in t if w])


CPU times: user 18.3 s, sys: 13.1 ms, total: 18.3 s
Wall time: 18.4 s


In [8]:
df[['text_clean']].sample(3)

,text_clean
716,"[пиротехник, истекш, срок, годност, продава, берез, декабр, белт, берез, правоохранител, изъя, индивидуальн, предпринимател, пиротехник, истекш, срок, годност, сообщ, белт, старш, инспектор, отдел..."
3088,"[департамент, межнациональн, отношен, министерств, региональн, разв, комиинформ, новост, госслужа, должн, осторожн, формулировк, особен, дел, каса, межэтническ, отношен, представител, минрегионраз..."
2585,"[риам, дек, полн, лун, помеша, москвич, жител, подмосков, наблюда, звездопад, геминид, ноч, декабр, сообщ, риам, вторник, представител, пресс, служб, московск, планетар, ран, сайт, планетар, сообщ..."
